In [ ]:
# expense_saas_generator_v2_fixed.py
# Generates two CSVs: accounts.csv and subscriptions.csv

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# ----------------------------
# CONFIG
# ----------------------------
np.random.seed(42)
random.seed(42)

START_YEAR = 2023
START_DATE = datetime(START_YEAR, 1, 1)
END_DATE = datetime(2025, 12, 1)
# END_DATE = datetime.now().replace(day=1)  # include the current month if you'd prefer
# NUM_ACCOUNTS now driven by monthly signups, so not used

PLANS = ["Starter", "Growth", "Pro", "Enterprise"]
# base monthly price per plan (base fee, seat price will be added)
PLAN_BASE = {"Starter": 19, "Growth": 49, "Pro": 149, "Enterprise": 499}
PLAN_SEAT_PRICE = {"Starter": 5, "Growth": 9, "Pro": 15, "Enterprise": 25}

# baseline probabilities (monthly)
BASE_UPGRADE_PROB = {
    "Starter": 0.10,   # 10% chance per month to upgrade
    "Growth": 0.06,
    "Pro": 0.02,
    "Enterprise": 0.00}

BASE_DOWNGRADE_PROB = {
    "Starter": 0.00,
    "Growth": 0.01,
    "Pro": 0.01,
    "Enterprise": 0.00}

# seat expansion (monthly chance) and expansion size distribution (multiplicative factor)
BASE_SEAT_EXPANSION_PROB = {"Starter": 0.02, "Growth": 0.05, "Pro": 0.08, "Enterprise": 0.10}
SEAT_EXPANSION_FACTOR_RANGE = (1.20, 1.75)  # +20% -> 75% (multiplicative on seats)

# plan-dependent base churn monthly probability (before modifiers)
PLAN_BASE_CHURN = {"Starter": 0.06, "Growth": 0.02, "Pro": 0.008, "Enterprise": 0.002}

# Business Size Categories (FINTECH SaaS realistic)
BUSINESS_SIZES = ["Micro (1-10)", "Small (11-50)", "Medium (51-200)", "Large (201-1000)"]

# Seat distributions (based on team size typical for expense management platforms)
BUSINESS_SIZE_SEAT_DIST = {
    "Micro (1-10)": (1, 10),        # 1–10 employees
    "Small (11-50)": (8, 45),       # 11–50 employees
    "Medium (51-200)": (30, 180),   # 51–200 employees
    "Large (201-1000)": (120, 800)  # 201–1000 employees
}

# Churn modifiers (small firms churn more)
BUSINESS_SIZE_CHURN_MOD = {
    "Micro (1-10)": 1.5,   # very small businesses churn more
    "Small (11-50)": 1.1,
    "Medium (51-200)": 0.85,
    "Large (201-1000)": 0.6    # large companies churn the least
}

# ----------------------------
# Add realistic industries and locations
# ----------------------------

INDUSTRIES = [
    "Technology",
    "E-commerce",
    "Professional Services",
    "Financial Services",
    "Healthcare",
    "Manufacturing",
    "Logistics & Transport",
    "Real Estate",
    "Hospitality",
    "Non-profit",
    "Retail"]

# Weighted because tech & services are most common for expense SaaS
INDUSTRY_WEIGHTS = [0.25, 0.10, 0.20, 0.15, 0.07, 0.05, 0.05, 0.05, 0.04, 0.02, 0.02]

LOCATIONS = [
    "United States",
    "Canada",
    "United Kingdom",
    "Germany",
    "France",
    "Netherlands",
    "Australia",
    "Singapore",
    "UAE",
    "South Africa",
    "India",]

LOCATION_WEIGHTS = [0.45, 0.08, 0.12, 0.06, 0.04, 0.03, 0.07, 0.03, 0.03, 0.02, 0.07]

# Interest rate hikes period (2022-01-01 to 2023-12-31) - modest effects
RATE_HIKE_START = datetime(2022, 1, 1)
RATE_HIKE_END = datetime(2023, 12, 31)
RATE_HIKE_CHURN_BOOST = 1.15     # 15% increase in churn probability
RATE_HIKE_DOWNGRADE_BOOST = 1.25 # downgrades more likely
RATE_HIKE_SEAT_EXPANSION_REDUCTION = 0.8  # seat expansions slightly lower

# signup seasonality: simple monthly weight (1 = normal)
MONTHLY_SIGNUP_WEIGHTS = [1.0 + 0.05 * np.sin(i / 12.0 * 3 * np.pi) for i in range(12)]

# distribution of initial plan on signup
PLAN_WEIGHTS = [0.55, 0.30, 0.12, 0.03]  # Starter most common

# ----------------------------
# Helpers
# ----------------------------
def month_iter(start_dt, end_dt):
    cur = start_dt.replace(day=1)
    while cur <= end_dt.replace(day=1):
        yield cur
        cur = (cur + pd.DateOffset(months=1))

def within_period(dt, start, end):
    return (dt >= start) and (dt <= end)

def compute_mrr_for_plan_seats(plan, seats):
    """Compute MRR given plan & seats"""
    base = PLAN_BASE[plan]
    seat_price = PLAN_SEAT_PRICE[plan]
    return round(base + seats * seat_price, 2)

# ----------------------------
# Generate Accounts over time (REALISTIC GROWTH)
# ----------------------------

accounts = []
account_id_counter = 1

def monthly_new_signups(current_month):
    """Return number of new signups for the month with macro effects."""
    base = 120  # average new signups per month for a growing fintech SaaS

    # seasonality
    seasonality = MONTHLY_SIGNUP_WEIGHTS[current_month.month - 1]

    # COVID dip
    covid_mult = COVID_SIGNUP_MULTIPLIER if (COVID_START <= current_month <= COVID_END) else 1.0

    # interest rate period (slightly fewer signups)
    rate_mult = 0.9 if (RATE_HIKE_START <= current_month <= RATE_HIKE_END) else 1.0

    # trend growth (company grows YoY)
    years_since_start = max((current_month.year - START_DATE.year), 0)
    trend_growth = 1 + 0.10 * years_since_start  # ~10% YoY growth

    # final expected rate
    signups = base * seasonality * covid_mult * rate_mult * trend_growth

    # Poisson noise around expectation
    return int(np.random.poisson(signups))

# Generate customers month by month
for month in month_iter(START_DATE, END_DATE):

    # 1. generate NEW signups this month
    n_new = monthly_new_signups(month)
    for _ in range(n_new):
        signup = month
        business_size = np.random.choice(BUSINESS_SIZES, p=[0.5, 0.3, 0.15, 0.05])

        # Seat range depending on size
        seat_min, seat_max = BUSINESS_SIZE_SEAT_DIST[business_size]
        seats = int(np.random.randint(seat_min, seat_max + 1))

        # Plan selection
        plan_choice = np.random.choice(PLANS, p=PLAN_WEIGHTS)
        if seats >= 50 and plan_choice == "Starter":
            plan_choice = "Growth"
        if seats >= 200 and plan_choice in ["Starter","Growth"]:
            plan_choice = "Pro"

        accounts.append({
            "account_id": account_id_counter,
            "signup_date": signup.date(),
            "churn_date": None,
            "plan_type": plan_choice,
            "seats": seats,
            "business_size": business_size,
            "industry": np.random.choice(INDUSTRIES, p=INDUSTRY_WEIGHTS),
            "hq_location": np.random.choice(LOCATIONS, p=LOCATION_WEIGHTS)
        })

        account_id_counter += 1

accounts_df = pd.DataFrame(accounts)

# enforce categorical dtype for business_size and validate values
accounts_df['business_size'] = pd.Categorical(
    accounts_df['business_size'],
    categories=BUSINESS_SIZES,
    ordered=True
)

# small validation print-out (will be empty if everything is correct)
invalid_sizes = accounts_df.loc[~accounts_df['business_size'].isin(BUSINESS_SIZES), 'business_size'].unique()
if len(invalid_sizes) > 0:
    print("Warning: unexpected business_size values:", invalid_sizes)

# ----------------------------
# Generate monthly subscription snapshots (with dynamics)
# ----------------------------
rows = []
for _, acc in accounts_df.iterrows():
    acc = acc.copy()  # mutable
    account_id = acc["account_id"]
    signup_dt = pd.to_datetime(acc["signup_date"])
    start_month = signup_dt.replace(day=1)

    churned = False
    churn_date = None

    current_plan = acc["plan_type"]
    current_seats = int(acc["seats"])
    months_alive = 0

    for month_dt in month_iter(start_month, END_DATE):
        if churned:
            break

        months_alive += 1

        # Determine dynamic probabilities for this month
        base_churn = PLAN_BASE_CHURN.get(current_plan, 0.02)
        size_mod = BUSINESS_SIZE_CHURN_MOD.get(acc["business_size"], 1.0)
        churn_prob = base_churn * size_mod

        # COVID & Rate-hike effects
        if within_period(month_dt, COVID_START, COVID_END):
            churn_prob *= COVID_CHURN_BOOST
        if within_period(month_dt, RATE_HIKE_START, RATE_HIKE_END):
            churn_prob *= RATE_HIKE_CHURN_BOOST

        # Prevent churn in first 3 months (typical onboarding immunity)
        if months_alive <= 3:
            churn_prob = 0.0

        # Upgrade / downgrade probabilities
        upgrade_prob = BASE_UPGRADE_PROB.get(current_plan, 0.0)
        downgrade_prob = BASE_DOWNGRADE_PROB.get(current_plan, 0.0)

        if within_period(month_dt, RATE_HIKE_START, RATE_HIKE_END):
            upgrade_prob *= 0.9
            downgrade_prob *= RATE_HIKE_DOWNGRADE_BOOST

        # Seat expansion chance for this plan
        seat_exp_prob = BASE_SEAT_EXPANSION_PROB.get(current_plan, 0.02)
        if within_period(month_dt, RATE_HIKE_START, RATE_HIKE_END):
            seat_exp_prob *= RATE_HIKE_SEAT_EXPANSION_REDUCTION

        # -------------------------
        # Event Decisions
        # -------------------------
        # Upgrade
        if current_plan != "Enterprise" and np.random.rand() < upgrade_prob:
            if current_plan == "Starter":
                current_plan = "Growth"
            elif current_plan == "Growth":
                current_plan = "Pro"
            elif current_plan == "Pro":
                current_plan = "Enterprise"
            seat_increase_pct = np.random.uniform(0.2, 1.0)
            add_seats = max(1, int(current_seats * seat_increase_pct))
            current_seats += add_seats

        # Downgrade
        if current_plan != "Starter" and np.random.rand() < downgrade_prob:
            if current_plan == "Enterprise":
                current_plan = "Pro"
            elif current_plan == "Pro":
                current_plan = "Growth"
            elif current_plan == "Growth":
                current_plan = "Starter"
            seat_decrease = max(0, int(current_seats * np.random.uniform(0.05, 0.3)))
            current_seats = max(1, current_seats - seat_decrease)

        # Seat expansion event
        if np.random.rand() < seat_exp_prob:
            factor = np.random.uniform(SEAT_EXPANSION_FACTOR_RANGE[0], SEAT_EXPANSION_FACTOR_RANGE[1])
            new_seats = max(current_seats + 1, int(current_seats * factor))
            # Correct, consistent label checks for caps
            if acc["business_size"] == "Micro (1-10)":
                new_seats = min(new_seats, 20)
            elif acc["business_size"] == "Small (11-50)":
                new_seats = min(new_seats, 80)
            elif acc["business_size"] == "Medium (51-200)":
                new_seats = min(new_seats, 300)
            elif acc["business_size"] == "Large (201-1000)":
                new_seats = min(new_seats, 1000)
            current_seats = new_seats

        # Churn decision
        if np.random.rand() < churn_prob:
            churned = True
            churn_date = month_dt.date()

        # Compute MRR
        mrr = compute_mrr_for_plan_seats(current_plan, current_seats)

        rows.append({
            "account_id": account_id,
            "month": month_dt.date(),
            "plan_type": current_plan,
            "seats": int(current_seats),
            "mrr": mrr
        })

        if churned:
            accounts_df.loc[accounts_df["account_id"] == account_id, "churn_date"] = churn_date
            break

# Build DataFrames and save
subs_df = pd.DataFrame(rows)

# Save to your desired paths
accounts_df.to_csv(r"C:\Users\GeoffreyRwamakuba\OneDrive - SpecFin Capital Ltd\Analyst\Geoffrey\1.Data Analysis+\accounts.csv", index=False)
subs_df.to_csv(r"C:\Users\GeoffreyRwamakuba\OneDrive - SpecFin Capital Ltd\Analyst\Geoffrey\1.Data Analysis+\subscriptions.csv", index=False)

print("Wrote accounts.csv ({} rows) and subscriptions.csv ({} rows)".format(
    len(accounts_df), len(subs_df)
))


Wrote accounts.csv (4665 rows) and subscriptions.csv (65024 rows)
